In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import fake_useragent
import re
import warnings
warnings.filterwarnings("ignore")

ua = fake_useragent.UserAgent()

In [2]:
def get_url(category, tot_pages):
    for page in range(0,tot_pages+1, 40):
        if page % log_frequency == 0:
            print(f'Log: working over page {page}...')       
        
        url_main = f'https://zonasporta.com/category/{category}/offset{page}'
        response = requests.get(url_main, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find_all(class_ = 'nazvanie_new')
        for item in data:
            item_url = 'https://zonasporta.com' + item.find('a').attrs['href']
            yield item_url

In [3]:
def get_item(category, tot_pages):
    
    for url_item in get_url(category, tot_pages):
        response = requests.get(url_item, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')
        
        try:
            title = soup.find('h1').text
        except:
            title = ''

        try:
            article = soup.find(class_ = 'pc-fleft mr5px').text
            article = article.replace(' ', '').replace('\n', '')
            article = article.replace('Артикул:', '')
        except:
            article = ''

        try:
            image = soup.find(class_ = 'pc-h pc-cursor')
            url = 'https://zonasporta.com' + image.attrs['src']

        except:
            print(f'Log: no image for {article} found...')
            url = ''
   
        try:
            info = soup.find(class_ = 'cpt_product_description')
            info = info.text.replace('\n', ' ').strip()
            desc = info.split('Характеристики:')[0].strip()
            char = info.split('Характеристики:')[1].strip()
        except:
            char = ''

        try:
            price = soup.find(class_ = 'totalPrice').text

        except:
            price = ''

        try:
            categories = soup.find(class_ = 'b_c')
            cat = categories.find_all(class_ = 'cat')
            cat_1 = cat[0].text.replace('\n', ' ').strip()
        except:
            cat_1 = ''
        try:
            cat_2 = cat[1].text.replace('\n', ' ').strip()
        except:
            cat_2 = ''
        try:
            cat_3 = cat[2].text.replace('\n', ' ').strip()
        except:
            cat_3 = ''
    
        yield title, article, price, cat_1, cat_2, cat_3, url, desc, char

In [4]:
df_columns = ['title', 'article','price', 'cat_1', 'cat_2', 'cat_3','url', 
              'description', 'caracteristics']

df = pd.DataFrame(columns = df_columns)
save_frequency = 400
log_frequency = 600

In [5]:
def to_csv(category, tot_pages):
    counter = 0  

    for item in get_item(category, tot_pages):
            data = []
            for i in range(len(df_columns)):
                data.append(item[i])
            df.loc[len(df.index)] = data
            #time.sleep(0.05)

            counter += 1
            if counter % save_frequency == 0:
                df.to_csv('zonasporta2.csv')
                print(f'{counter} items saved in csv..')
    df.to_csv('zonasporta2.csv')

    print(f'\nTotal items saved: {counter}')  

In [6]:
to_csv('begovye-dorozhki', 1560)

Log: working over page 0...
Log: no image for  found...
400 items saved in csv..
Log: no image for 55178 found...
Log: no image for  found...
Log: no image for  found...
Log: working over page 600...
Log: no image for  found...
Log: no image for  found...
Log: no image for  found...
Log: no image for  found...
Log: no image for  found...
Log: no image for  found...
800 items saved in csv..
Log: no image for  found...
Log: no image for  found...
Log: no image for  found...
Log: no image for  found...
Log: no image for  found...
1200 items saved in csv..
Log: working over page 1200...
Log: no image for  found...
Log: no image for  found...
Log: no image for  found...
Log: no image for  found...
Log: no image for  found...
Log: no image for  found...
Log: no image for  found...
Log: no image for  found...

Total items saved: 1594


In [7]:
to_csv('steppery', 40)

Log: working over page 0...

Total items saved: 71


In [9]:
df[df['price'] == '']

,title,article,price,cat_1,cat_2,cat_3,url,description,caracteristics
39,Беговая дорожка Carbon Fitness T608 Slim,,,,Беговые дорожки,Carbon Fitness,,Купить Беговую дорожку Carbon Fitness T220 в н...,
461,Беговая дорожка Optima Fitness Solo,,,,Беговые дорожки,Sole Fitness,,Купить беговую дорожку Sole Fitness F65 (2016)...,
490,,,,,Беговые дорожки,DFC,,Купить беговую дорожку DFC Ascona T595 в нашем...,
651,Беговая дорожка X-Trend LT-2203,,,,Беговые дорожки,X-Trend,,Купить беговую дорожку X-Trend LT-2301 в нашем...,
676,Беговая дорожка LifeSpan TR8000i,,,,Беговые дорожки,BH Fitness,,Купить беговую дорожку BH Fitness F1 Smart G64...,
680,Беговая дорожка Laufstein Enigma,,,,Беговые дорожки,Laufstein,,Купить беговую дорожку Laufstein Gemini – в на...,
718,Беговая дорожка Aerofit MaxFit 18,,,,Беговые дорожки,AeroFIT,,Купить беговую дорожку Aerofit 9900T – в наше...,
726,Беговая дорожка Sport Elite SE-1611 министадион,,,,Беговые дорожки,Body Sculpture,,Купить беговую дорожку Body Sculpture ВТ-6010 ...,
727,Беговая дорожка Body Sculpture ВТ-3131S2,,,,Беговые дорожки,Body Sculpture,,Купить беговую дорожку Body Sculpture ВТ-6010 ...,
815,Тренажер для ходьбы Bowflex TreadClimber TC20,,,,Беговые дорожки,Svensson Body Labs,,Купить беговую дорожку Svensson Body Labs Orth...,
